# Hackathon with WDI

## Building a front-end app with machine learning to predict taxi information

In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib as plt

import seaborn as sns

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (10,6)

%matplotlib inline

In [2]:
# Loading in cleaned df from capstone project

with open('/Users/peidacai/Desktop/taxi_df_dropoff_in_nyc_only.pkl', 'r') as f:
    df = pickle.load(f)

In [3]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,...,improvement_surcharge,total_amount,trip_month,day_of_week,pickup_hour,dropoff_hour,trip_duration,Avg_spd,dropoff_point,zipcode
0,2015-07-01 00:00:20,2015-07-01 00:07:57,1,2.10,-73.983780,40.755219,1,-73.987389,40.733372,1,...,0.3,11.76,7,2,0,0,7.0,18.000000,"[-73.9873886108, 40.7333717346]",10003
1,2015-07-01 00:00:42,2015-07-01 00:08:27,2,1.18,-74.003471,40.725307,1,-74.004333,40.737984,1,...,0.3,10.38,7,2,0,0,7.0,10.114286,"[-74.0043334961, 40.7379837036]",10014
2,2015-07-01 00:01:06,2015-07-01 00:12:06,1,3.10,-74.003044,40.717903,1,-73.958641,40.712761,1,...,0.3,15.70,7,2,0,0,11.0,16.909091,"[-73.9586410522, 40.7127609253]",11211
3,2015-07-01 00:01:29,2015-07-01 00:07:52,2,2.10,-73.982910,40.726765,1,-73.967339,40.752625,1,...,0.3,11.15,7,2,0,0,6.0,21.000000,"[-73.967338562, 40.7526245117]",10017
4,2015-07-01 00:01:53,2015-07-01 00:08:35,1,1.24,-74.002365,40.739746,1,-74.010254,40.729797,2,...,0.3,8.30,7,2,0,0,6.0,12.400000,"[-74.0102539062, 40.7297973633]",10014


In [4]:
df.columns

Index([u'tpep_pickup_datetime', u'tpep_dropoff_datetime', u'passenger_count',
       u'trip_distance', u'pickup_longitude', u'pickup_latitude',
       u'RatecodeID', u'dropoff_longitude', u'dropoff_latitude',
       u'payment_type', u'fare_amount', u'extra', u'mta_tax', u'tip_amount',
       u'tolls_amount', u'improvement_surcharge', u'total_amount',
       u'trip_month', u'day_of_week', u'pickup_hour', u'dropoff_hour',
       u'trip_duration', u'Avg_spd', u'dropoff_point', u'zipcode'],
      dtype='object')

In [60]:
credit_pay = df[df['payment_type'] == 1]
credit_pay['tip_percent'] = 100*(credit_pay['tip_amount'] / \
(credit_pay['total_amount']- credit_pay['tip_amount']))
credit_pay.head()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,...,total_amount,trip_month,day_of_week,pickup_hour,dropoff_hour,trip_duration,Avg_spd,dropoff_point,zipcode,tip_percent
0,2015-07-01 00:00:20,2015-07-01 00:07:57,1,2.10,-73.983780,40.755219,1,-73.987389,40.733372,1,...,11.76,7,2,0,0,7.0,18.000000,"[-73.9873886108, 40.7333717346]",10003,20.000000
1,2015-07-01 00:00:42,2015-07-01 00:08:27,2,1.18,-74.003471,40.725307,1,-74.004333,40.737984,1,...,10.38,7,2,0,0,7.0,10.114286,"[-74.0043334961, 40.7379837036]",10014,25.060241
2,2015-07-01 00:01:06,2015-07-01 00:12:06,1,3.10,-74.003044,40.717903,1,-73.958641,40.712761,1,...,15.70,7,2,0,0,11.0,16.909091,"[-73.9586410522, 40.7127609253]",11211,18.045113
3,2015-07-01 00:01:29,2015-07-01 00:07:52,2,2.10,-73.982910,40.726765,1,-73.967339,40.752625,1,...,11.15,7,2,0,0,6.0,21.000000,"[-73.967338562, 40.7526245117]",10017,19.892473
5,2015-07-01 00:03:28,2015-07-01 00:06:54,1,0.46,-73.971519,40.762791,1,-73.973671,40.757149,1,...,6.96,7,2,0,0,3.0,9.200000,"[-73.9736709595, 40.7571487427]",10022,20.000000


In [61]:
df1 = credit_pay.copy()

## Taxi drop off hour

In [62]:
df1.head(1)

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,...,total_amount,trip_month,day_of_week,pickup_hour,dropoff_hour,trip_duration,Avg_spd,dropoff_point,zipcode,tip_percent
0,2015-07-01 00:00:20,2015-07-01 00:07:57,1,2.1,-73.98378,40.755219,1,-73.987389,40.733372,1,...,11.76,7,2,0,0,7.0,18.0,"[-73.9873886108, 40.7333717346]",10003,20.0


In [108]:
df1[['total_amount', 'tip_amount', 'trip_distance', 'Avg_spd', 'tip_percent']].describe()

,total_amount,tip_amount,trip_distance,Avg_spd,tip_percent
count,804493.000000,804493.000000,804493.000000,804493.000000,804493.000000
mean,16.034631,2.562024,2.674841,12.377427,19.334584
std,10.123903,1.840682,2.532208,6.116262,7.129225
min,3.900000,0.100000,0.010000,0.005357,0.159134
25%,9.360000,1.450000,1.100000,8.181818,17.241379
50%,12.950000,2.000000,1.800000,11.142857,20.000000
75%,18.590000,3.000000,3.170000,15.000000,20.000000
max,210.300000,20.000000,14.990000,63.000000,2000.000000


In [109]:
taxi_dropoff_hr = pd.pivot_table(df, values = ['total_amount', 'tip_amount', 'trip_distance', 'Avg_spd'], index = ['dropoff_hour'])

In [110]:
taxi_dropoff_hr = taxi_dropoff_hr.applymap(lambda x: round(x, 2))

In [111]:
taxi_dropoff_hr = pd.concat([taxi_dropoff_hr, pd.pivot_table(df, values = 'total_amount', index = ['dropoff_hour'], aggfunc=len)], axis = 1)
taxi_dropoff_hr.head()

,Avg_spd,tip_amount,total_amount,trip_distance,total_amount
dropoff_hour,,,,,
0,15.46,1.94,17.43,3.60,53383.0
1,16.24,1.83,16.85,3.55,39306.0
2,16.78,1.69,16.07,3.43,28362.0
3,17.54,1.61,16.20,3.55,20352.0
4,19.46,1.69,17.99,4.14,15607.0


In [112]:
taxi_dropoff_hr.loc[:, 'total_amount'] = taxi_dropoff_hr['total_amount'].astype(int)

In [113]:
taxi_dropoff_hr.columns = ['Avg_spd', 'tip_amount', 'total_amount', 'trip_distance', 'trip_count']

In [10]:
taxi_dropoff_hr.to_json('/Users/peidacai/Desktop/taxi_dropoff_hr.json', orient = 'index')

## Taxi pickup hour

In [11]:
taxi_pickup_hr = pd.pivot_table(df, values = ['total_amount', 'tip_amount', 'trip_distance', 'Avg_spd'], index = ['pickup_hour'])

In [12]:
taxi_pickup_hr = taxi_pickup_hr[['Avg_spd', 'tip_amount', 'total_amount', 'trip_distance']].applymap(lambda x: round(x, 2))
taxi_pickup_hr.head()

,Avg_spd,tip_amount,total_amount,trip_distance
pickup_hour,,,,
0,15.59,1.84,16.69,3.44
1,16.29,1.71,15.92,3.33
2,16.87,1.62,15.63,3.32
3,17.74,1.57,16.01,3.50
4,20.25,1.78,18.92,4.39


In [13]:
taxi_pickup_hr = pd.concat([taxi_pickup_hr, pd.pivot_table(df, values = 'total_amount', index = ['pickup_hour'], aggfunc=len)], axis = 1)
taxi_pickup_hr.columns = ['Avg_spd', 'tip_amount', 'total_amount', 'trip_distance', 'trip_count']
taxi_pickup_hr.head()

,Avg_spd,tip_amount,total_amount,trip_distance,trip_count
pickup_hour,,,,,
0,15.59,1.84,16.69,3.44,49415.0
1,16.29,1.71,15.92,3.33,36264.0
2,16.87,1.62,15.63,3.32,26524.0
3,17.74,1.57,16.01,3.50,19222.0
4,20.25,1.78,18.92,4.39,14558.0


In [14]:
taxi_pickup_hr.loc[:, 'trip_count'] = taxi_pickup_hr['trip_count'].astype(int)

In [15]:
import json

In [16]:
taxi_pickup_hr.to_json('/Users/peidacai/Desktop/taxi_pickup_hr.json', orient = 'index')

## Taxi drop off by day

In [17]:
taxi_dropoff_day = pd.pivot_table(df, values = ['total_amount', 'tip_amount', 'trip_distance', 'Avg_spd'], index = ['day_of_week'])
taxi_dropoff_day = taxi_dropoff_day.applymap(lambda x: round(x, 2))
taxi_dropoff_day = pd.concat([taxi_dropoff_day, pd.pivot_table(df, values = 'total_amount', index = ['day_of_week'], aggfunc=len)], axis = 1)
taxi_dropoff_day.columns = ['Avg_spd', 'tip_amount', 'total_amount', 'trip_distance', 'trip_count']
taxi_dropoff_day.to_json('/Users/peidacai/Desktop/taxi_dropoff_day.json', orient = 'index')

# Predictive modeling

In [6]:
from sklearn.preprocessing import Normalizer, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.feature_selection import SelectPercentile
from sklearn.pipeline import make_pipeline

In [7]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,...,improvement_surcharge,total_amount,trip_month,day_of_week,pickup_hour,dropoff_hour,trip_duration,Avg_spd,dropoff_point,zipcode
0,2015-07-01 00:00:20,2015-07-01 00:07:57,1,2.10,-73.983780,40.755219,1,-73.987389,40.733372,1,...,0.3,11.76,7,2,0,0,7.0,18.000000,"[-73.9873886108, 40.7333717346]",10003
1,2015-07-01 00:00:42,2015-07-01 00:08:27,2,1.18,-74.003471,40.725307,1,-74.004333,40.737984,1,...,0.3,10.38,7,2,0,0,7.0,10.114286,"[-74.0043334961, 40.7379837036]",10014
2,2015-07-01 00:01:06,2015-07-01 00:12:06,1,3.10,-74.003044,40.717903,1,-73.958641,40.712761,1,...,0.3,15.70,7,2,0,0,11.0,16.909091,"[-73.9586410522, 40.7127609253]",11211
3,2015-07-01 00:01:29,2015-07-01 00:07:52,2,2.10,-73.982910,40.726765,1,-73.967339,40.752625,1,...,0.3,11.15,7,2,0,0,6.0,21.000000,"[-73.967338562, 40.7526245117]",10017
4,2015-07-01 00:01:53,2015-07-01 00:08:35,1,1.24,-74.002365,40.739746,1,-74.010254,40.729797,2,...,0.3,8.30,7,2,0,0,6.0,12.400000,"[-74.0102539062, 40.7297973633]",10014


In [63]:
df1 = df1[df1['tip_amount'] >= 0.1]
df1 = df1[df1['tip_amount'] <=  20]

In [64]:
df1 = df1[df1['total_amount'] > 0]

In [65]:
df1 = df1[df1['trip_distance'] < 15]

### Defining X and y

In [67]:
zip_dum = pd.get_dummies(df1['zipcode'], prefix='Zip')

In [68]:
zip_dum.head()

,Zip_10001,Zip_10002,Zip_10003,Zip_10004,Zip_10005,Zip_10006,Zip_10007,Zip_10009,Zip_10010,Zip_10011,...,Zip_11429,Zip_11430,Zip_11432,Zip_11433,Zip_11434,Zip_11435,Zip_11436,Zip_11691,Zip_11694,Zip_11697
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
# Drop zipcode of 10001
# Need to have a separate handler for zipcode with 10001

del zip_dum['Zip_10001']

In [95]:
X_zip_dum = zip_dum.iloc[0,:]

with open('/Users/peidacai/Desktop/X_zip_dum.pkl', 'w') as f:
    pickle.dump(X_zip_dum, f)

In [70]:
# Hour 0 is dropped

hour_dum = pd.get_dummies(df1['pickup_hour'], prefix='hour')

In [93]:
X_hour_dum = hour_dum.iloc[0,:]

In [94]:
with open('/Users/peidacai/Desktop/X_hour_dum.pkl', 'w') as f:
    pickle.dump(X_hour_dum, f)

In [71]:
hour_dum.head()

,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [72]:
del hour_dum['hour_0']

In [73]:
# Sunday is dropped

dow_dum = pd.get_dummies(df1['day_of_week'], prefix='dow')

In [74]:
dow_dum.head()

,dow_0,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6
0,0,0,1,0,0,0,0
1,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0
5,0,0,1,0,0,0,0


In [75]:
del dow_dum['dow_0']

In [96]:
X_dow_dum = dow_dum.iloc[0,:]

with open('/Users/peidacai/Desktop/X_dow_dum.pkl', 'w') as f:
    pickle.dump(X_dow_dum, f)

In [101]:
X = pd.concat([hour_dum, dow_dum, zip_dum], axis =1)
y = df1['tip_percent']

In [102]:
X.shape

(804493, 207)

In [34]:
with open('/Users/peidacai/Desktop/X.pkl', 'w') as f:
    pickle.dump(X, f)

## Modeling

In [41]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from sklearn.model_selection import cross_val_score, train_test_split

In [39]:
rand = 44

In [42]:
# Regressors models

rfr = RandomForestRegressor(random_state=rand)
#etr = ExtraTreesRegressor(random_state=rand)
#adar = AdaBoostRegressor(random_state=rand)
gbr = GradientBoostingRegressor(random_state=rand)
linr = LinearRegression()
#knnr = KNeighborsRegressor()
#svr = SVR()

In [153]:
# List of different scaling methods
score = cross_val_score(gbr, X, y, cv =3, scoring = 'neg_mean_squared_error')

In [154]:
np.sqrt(abs(score.mean()))

7.1289422991295703

In [155]:
# List of different scaling methods
score = cross_val_score(rfr, X, y, cv =3, scoring = 'neg_mean_squared_error')

In [156]:
np.sqrt(abs(score.mean()))

7.2316324829686618

In [157]:
# List of different scaling methods
score = cross_val_score(linr, X, y, cv =3, scoring = 'neg_mean_squared_error')

In [158]:
np.sqrt(abs(score.mean()))

7.1279707385945681

0          20.000000
1          25.060241
2          18.045113
3          19.892473
5          20.000000
7          25.000000
8          20.000000
10         14.563107
13         20.000000
14         20.000000
15         30.000000
16          8.474576
21         19.907407
22         20.000000
23         29.787234
24          4.694836
25         16.853933
26         25.000000
28         20.000000
29         19.918699
30         20.000000
31         30.000000
32         19.942197
33         20.000000
34         12.781955
35         19.879518
36          7.246377
38         52.083333
39          2.040816
42         12.970169
             ...    
1330693    19.907407
1330694    12.048193
1330695    20.000000
1330699    20.000000
1330701    15.822785
1330704    19.927536
1330705    19.871795
1330706    20.000000
1330708    20.000000
1330709    13.793103
1330711    25.000000
1330712    20.000000
1330713    19.927536
1330714    10.000000
1330715    13.698630
1330717    18.072289
1330718    20

In [160]:
clf = linr

In [161]:
clf.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [164]:
X_test = X.head(1)

In [167]:
clf.predict(X_test)

array([ 19.38008345])

In [171]:
X_test

KeyError: 0

In [152]:
with open('/Users/peidacai/Desktop/tips_model.pkl', 'w') as f:
    pickle.dump(clf, f)

In [172]:
y = df1['Avg_spd']

In [174]:
clf.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [175]:
with open('/Users/peidacai/Desktop/spd_model.pkl', 'w') as f:
    pickle.dump(clf, f)

In [50]:
y = df1['total_amount']

In [51]:
clf.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [52]:
with open('/Users/peidacai/Desktop/fare.pkl', 'w') as f:
    pickle.dump(clf, f)

## Restricted df1 to only required columns

In [114]:
df2 = df1[['Avg_spd', 'tip_percent', 'total_amount', 'trip_distance', 'zipcode', 'pickup_hour', 'day_of_week']]

In [115]:
df2['Avg_spd'].mean()

12.377427093820938

In [176]:
df2.shape

(804493, 7)

In [177]:
with open('/Users/peidacai/Desktop/small_df.pkl', 'w') as f:
    pickle.dump(df2,f)

In [119]:
d['Avg_spd'] = 12

In [120]:
d

{'Avg_spd': 12}

In [128]:
df2.head()

,Avg_spd,tip_percent,total_amount,trip_distance,zipcode,pickup_hour,day_of_week
0,18.000000,20.000000,11.76,2.10,10003,0,2
1,10.114286,25.060241,10.38,1.18,10014,0,2
2,16.909091,18.045113,15.70,3.10,11211,0,2
3,21.000000,19.892473,11.15,2.10,10017,0,2
5,9.200000,20.000000,6.96,0.46,10022,0,2


In [178]:
df2['zipcode'].nunique()

179

In [181]:
pd.pivot_table(df2, values = ['zipcode', 'pickup_hour', 'day_of_week'], index = ['Avg_spd', 'tip_percent', 'total_amount', 'trip_distance'])

day_of_week  pickup_hour  \
Avg_spd   tip_percent total_amount trip_distance                             
0.005357  20.000000   63.36        0.01                   6.0         14.0   
0.010909  30.000000   11.70        0.01                   2.0          5.0   
0.012371  13.636364   60.00        0.02                   4.0         13.0   
0.015385  30.000000   30.29        0.01                   4.0         23.0   
0.066667  20.000000   9.96         0.01                   4.0         20.0   
0.085714  20.000000   8.16         0.01                   3.0         12.0   
0.100000  20.000000   8.16         0.01                   4.0         20.0   
          25.040650   15.38        0.03                   4.0          9.0   
0.120000  17.241379   6.80         0.01                   4.0         15.0   
0.127273  14.184397   80.50        0.07                   2.0         12.0   
0.150000  20.000000   7.56         0.01                   1.0         18.0   
                      78.36        0.01                   2.0         16.0   
0.171429  19.953052   25.55        0.10                   6.0          0.0   
0.182609  20.000000   17.76        0.07                   3.0         12.0   
0.189474  7.812500    13.80        0.06                   5.0         23.0   
0.200000  15.873016   7.30         0.02                   5.0         13.0   
          20.000000   6.96         0.01                   3.0         18.0   
                      7.56         0.02                   6.0         12.0   
                      12.96        0.05                   1.0         11.0   
          30.000000   8.84         0.02                   4.0          1.0   
0.214286  20.000000   12.36        0.05                   2.0          9.0   
0.225000  20.000000   9.36         0.03                   3.0         22.0   
0.260870  19.918699   14.75        0.10                   1.0         22.0   
          19.932432   17.75        0.10                   1.0         14.0   
0.272727  20.000000   10.56        0.05                   0.0         14.0   
0.292941  10.000000   8.03         0.83                   3.0         21.0   
0.300000  7.518797    14.30        0.09                   2.0         18.0   
          13.698630   8.30         0.03                   3.0         17.0   
          20.000000   13.56        0.08                   0.0         12.0   
          30.000000   6.89         0.01                   2.0         18.0   
...                                                       ...          ...   
55.200000 19.934641   18.35        4.60                   6.0          4.0   
          20.000000   6.36         0.92                   5.0         14.0   
                      6.96         0.92                   0.0         20.0   
                      7.56         0.92                   1.0         19.0   
55.500000 19.811321   6.35         3.70                   6.0         19.0   
55.800000 15.000000   60.72        0.93                   1.0         18.0   
          20.000000   6.36         0.93                   0.0          6.0   
                      6.96         0.93                   2.0         21.0   
          25.000000   7.25         0.93                   0.0          1.0   
56.400000 12.068966   6.50         0.94                   5.0          4.0   
          24.905660   6.62         0.94                   5.0          7.0   
          30.000000   6.89         0.94                   5.0         14.0   
56.600000 20.000000   63.36        2.83                   5.0          7.0   
57.000000 19.791667   5.75         1.90                   5.0         22.0   
          30.000000   6.89         0.95                   6.0          6.0   
57.200000 8.570449    63.34        2.86                   4.0          7.0   
          20.003428   70.01        2.86                   0.0         16.0   
57.600000 7.625826    42.34        9.60                   4.0          8.0   
          20.000000   6.36         0.96                   6.0          9

In [148]:
pd.pivot_table()

(804493, 7)

In [185]:
# function to return mean features from dataframe based on user input

def mean_features(zipp, dow, hour):
    
    # spit out mean speed, mean trip distance, mean fare, mean tips percent
    # Filtering to subset dataframe
    dfo = df2[df2['zipcode'] == int(zipp)]
    dfo = dfo[dfo['pickup_hour'] == int(hour)]
    dfo = dfo[dfo['day_of_week'] == int(dow)]
    
    outdict = {}
    
    outdict['Avg_spd'] = round(dfo['Avg_spd'].mean(), 2)
    outdict['tip_amount'] = round(dfo['tip_percent'].mean(), 2)
    outdict['total_amount'] = round(dfo['total_amount'].mean(), 2)
    outdict['trip_distance'] = round(dfo['trip_distance'].mean(),2)
    outlist = []
    outlist.append(outdict)
        
    return outlist

In [184]:
[{'a':'1'}] + [{'b':'1'}]

[{'a': '1'}, {'b': '1'}]

In [141]:
import pprint as pprint

In [186]:
pprint.pprint(mean_features('10004', '2', '4'))

[{'Avg_spd': 19.09,
  'tip_amount': 20.78,
  'total_amount': 13.24,
  'trip_distance': 2.17}]


In [146]:
pprint.pprint(mean_features('11206', '2', '4'))

[{'Avg_spd': 20.53,
  'tip_amount': 23.15,
  'total_amount': 17.41,
  'trip_distance': 3.68}]


In [145]:
mean_features('10004', '2', '4')

[{'Avg_spd': 19.09,
  'tip_amount': 20.78,
  'total_amount': 13.24,
  'trip_distance': 2.17}]

In [136]:
a[0]

'trip_distance'